In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import sys
from pathlib import Path
import os
import pandas as pd

project_root = os.getenv('PROJECT_ROOT')
sys.path.append(str(project_root))
print(project_root)
print(os.getcwd())

/Users/ivang/PycharmProjects/pythonProject/
/Users/ivang/PycharmProjects/pythonProject/notebooks


# Power analysis for required sample size for Likert scale

In [5]:
import statsmodels.stats.power as smp

# Define the effect size as the correlation coefficient (r)
effect_size = 0.5  # for example, a medium effect size

# Significance level (alpha)
alpha = 0.1

# Desired power
power = 0.8

# Calculate the required sample size
analysis = smp.NormalIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='two-sided')

print(f"Required sample size: {sample_size}")

Required sample size: 49.45789459567311


# Load results

In [6]:
embedding_model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")

You try to use a model that was created with version 3.0.1, however, your version is 3.0.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/Users/ivang/miniconda3/envs/openml-tags/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
import json

with open('models/topic_model_text_generation_4/topics.json') as f:
    topics_json = json.load(f)

topics_json 

{'topic_representations': {'0': [['Themes: Binarized Regression Problem, Chemistry, Life Science, Mythbusting, Study, Target Feature, Nominal Classification; Overarching themes: Data Analysis, Research, Science, Mathematics',
    1]],
  '1': [['Themes: Anal Cat Data, Binarization, Chemistry, Life Science, Myth Busting, Study; Overarching themes: Data Analysis, Research, Classification',
    1]],
  '2': [['Themes: COVID-19, Pandemic, Cases, Deaths, Data Analysis, Visualization, Vaccinations, Demographics, Economics, Machine Learning, Statistics, Mobility, Tests, Healthcare, Length of Stay, Safety Behavior, No-Show Problem, Education Disruption; Overarching themes: Health, Science, Technology, Society, Policy',
    1]],
  '3': [['Themes: analcatdata, datasets, book, education, research, statistics, manufacturing, automotive accessories, hardware, bolts, counting, electronic eye, sensitivity setting, machine adjustments, time, data formats, comma-delimited, Microsoft Excel, quotation mark

In [8]:
from bertopic import BERTopic

loaded_model = BERTopic.load("models/topic_model_text_generation_llama2", embedding_model=embedding_model)

In [9]:
import numpy as np

embeddings = np.load('embeddings_prompts.npy')

In [70]:
import pandas as pd

df = pd.read_csv("data/prompt_descriptions.csv")

In [11]:
df

,prompt_description
0,The Annealing dataset from UCI contains inform...
1,"Dataset: kr-vs-kp\n\nTags: mythbusting, OpenML..."
2,"Labor Dataset\n\nTags: mythbusting, study, tes..."
3,"Arrhythmia Dataset\nTags: sport, study, UCI\n\..."
4,Letter Image Recognition Data\n\nDataset Tags:...
...,...
5213,Saugeen River Daily\n\nTags: Mean daily flow i...
5214,US Births\nTags: Number of Births in the Unite...
5215,Solar Power Tags: Solar Power Production\n\nTh...
5216,Wind Power Tags: Wind power production in mega...


In [12]:
data = df['prompt_description'].tolist()

In [13]:
topics, probs = loaded_model.transform(data, embeddings)

2024-07-12 00:30:54,868 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


In [14]:
topics

array([315, 525, 823, ..., 315, 500, 975])

In [15]:
loaded_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,16,0_Here is the extracted theme information:\n\n...,"[{'Themes': [['QSAR', 1], ['Mitogen-Activated ...",NaN
1,1,14,1_Here is the extracted and synthesized human-...,"[{'Themes': [['Health', 1], ['Medicine', 1], [...",NaN
2,2,14,2_Here is the extracted theme information:\n\n...,"[{'Themes': [['QSAR', 1], ['Drug Target', 1], ...",NaN
3,3,16,3_Here is the extracted theme information:\n\n...,"[{'Themes': [['QSAR', 1], ['Drug Target', 1], ...",NaN
4,4,13,4_Here is the extracted and synthesized human-...,"[{'Themes': [['Adult', 1], ['Income', 1], ['De...",NaN
...,...,...,...,...,...
1011,1011,4,1011_Here is the extracted and synthesized hum...,"[{'Themes': [['Machine Learning', 1], ['Cause-...",NaN
1012,1012,3,1012_Here is the extracted and synthesized hum...,"[{'Themes': [['Workforce', 1], ['Wildcat Strik...",NaN
1013,1013,3,1013_Here is the extracted and synthesized hum...,"[{'Themes': [['QSAR', 1], ['Protein Kinase', 1...",NaN
1014,1014,4,1014_Here is the extracted theme information i...,"[{'Themes': [['QSAR', 1], ['Drug Target', 1], ...",NaN


# TEMPORARY

In [16]:
documents = pd.DataFrame({"Document": data, "Topic": loaded_model.topics_})
repr_docs_mappings, _, _, _ = loaded_model._extract_representative_docs(
            loaded_model.c_tf_idf_,
            documents,
            loaded_model.topic_representations_,
            nr_repr_docs=30
        )
topic = 270
docs = repr_docs_mappings[topic]
labels = sorted(list(documents.Topic.unique()))
labels = [int(label) for label in labels]
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
_, words = loaded_model._c_tf_idf(documents_per_topic)
top_n_words = max(loaded_model.top_n_words, 30)
indices = loaded_model._top_n_idx_sparse(loaded_model.c_tf_idf_, top_n_words)
scores = loaded_model._top_n_values_sparse(loaded_model.c_tf_idf_, indices)
sorted_indices = np.argsort(scores, 1)
indices = np.take_along_axis(indices, sorted_indices, axis=1)
scores = np.take_along_axis(scores, sorted_indices, axis=1)

# Get top 30 words per topic based on c-TF-IDF score
topics_ctfidf = {label: [(words[word_index], score)
                    if word_index is not None and score > 0
                    else ("", 0.00001)
                    for word_index, score in zip(indices[index][::-1], scores[index][::-1])
                    ]
            for index, label in enumerate(labels)}
topics_ctfidf

{0: [('mitogen', 0.028212930528709605),
  ('kinase', 0.025864940435726416),
  ('activated', 0.021487892982856284),
  ('ste', 0.016433439490661516),
  ('tkl', 0.013321197372321443),
  ('cascade', 0.011327404102882277),
  ('mlk', 0.011253096657705415),
  ('mapk', 0.009907394916671386),
  ('protein', 0.009122256883059938),
  ('lineage', 0.00880691418105069),
  ('cmgc', 0.008525138182684949),
  ('cyclin', 0.008092750133581633),
  ('map', 0.007411974771032833),
  ('phosphorylation', 0.007363725901396394),
  ('binding', 0.007185524287529491),
  ('mekk', 0.006700304549489829),
  ('cdc', 0.006521753204962221),
  ('erk', 0.006468254906874727),
  ('mixed', 0.006263731458718263),
  ('group', 0.005708587693056891),
  ('cdk', 0.005560611319617099),
  ('jnk', 0.005560611319617099),
  ('enzyme', 0.005231341167800683),
  ('mitotic', 0.004980153928357928),
  ('1024bit', 0.004744798074166735),
  ('dependent', 0.004702089331985264),
  ('atp', 0.004631535601557312),
  ('regulation', 0.00456466467013799),


In [17]:
prompt = '''Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Also, don't give very similar tags/keywords, e.g. "Wine" and "Red Wine", just give one or the other in these cases. Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". Another important rule to obey - place more focus on the dataset names for theme extraction. And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Also, extract the theme of the text, what it is about, instead of the type of problem it is, for instance we don't care about "Regression", "Numerical Features", "Data Analysis", "Data", "Outliers", "Subsampling" or things of that sort, but we care about the ESSENCE of the text! Say {"Themes": [...], "Overarching themes": [...]} and give your answer in JSON format.
For example, for this text:
Text 1: The Biden administration is preparing to roll out a sweeping border executive action as early as Tuesday, according to two sources familiar with the discussions, who cautioned that timing is fluid.

White House officials have begun reaching out to mayors who represent cities along the US southern border to potentially join President Joe Biden when he announces the order, two other sources familiar with those conversations said.

For weeks, administration officials have been working through an executive action that would dramatically limit migrants’ ability to seek asylum at the US southern border — part of a strategy to try to give Biden the upper hand on one of his Republican rival’s key campaign issues. The action is designed to potentially blunt Republican attacks on border security and preempt former President Donald Trump ahead of the first presidential debate, which will be held on June 27 on CNN.
---
Text 2: Now that a New York jury has convicted former President Donald Trump of all 34 felony charges of falsifying business records, the next obvious question is: Can a convicted felon run for president?

Definitely.

Trump meets all three requirements. There is, arguably, another criterion laid out in the 14th Amendment, where it states that no one who has previously taken an oath of office who engages in insurrection can be an officer of the US. But the US Supreme Court ruled earlier this year that Congress would have to pass a special law invoking this prohibition. That’s not happening any time soon.

Judge Juan Merchan has scheduled Trump’s sentencing for July 11, which happens to be four days before the start of the Republican National Convention that is scheduled to take place in Milwaukee.
a
It is technically possible, although perhaps unlikely for a first-time offender, that Trump could be sentenced to prison time.
---
This would be your answer:
{"Themes": ["Biden Administration", "Border", "Executive Action", "Asylum", "Immigration", "Trump", "Felony", "Business Records", "Presidential Campaign", "Republican", "Debate", "Former President", "Conviction", "Sentencing", "Prison", "14th Amendment", "Insurrection", "Supreme Court", "Republican National Convention"], "Overarching themes": ["Politics", "Government", "Law", "Justice", "Elections"]}
---
Now, the above was just an example. Now, do it for the following text(s), be concise!:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]
---
Remember, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Also, don't give very similar tags/keywords, e.g. "Wine" and "Red Wine", just give one or the other in these cases. Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". Another important rule to obey - place more focus on the dataset names for theme extraction. And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Also, extract the theme of the text, what it is about, instead of the type of problem it is, for instance we don't care about "Regression", "Numerical Features", "Data Analysis", "Data", "Outliers", "Subsampling" or things of that sort, but we care about the ESSENCE of the text! Say {"Themes": [...], "Overarching themes": [...]} and give your answer in JSON format.
'''

In [18]:
def replace_documents(prompt, docs):
    to_replace = ""
    # for doc in docs:
    #     to_replace += f"- {doc}\n---\n"
    # same, but say Text 1: ... (newline) Text 2: ... (newline) etc.
    for i, doc in enumerate(docs):
        to_replace += f"Text {i+1}: {doc}\n---\n"
    prompt = prompt.replace("[DOCUMENTS]", to_replace)
    return prompt

In [19]:
def create_prompt(docs, topic, topics, prompt):
    keywords = list(zip(*topics[topic]))[0]
    if "[KEYWORDS]" in prompt:
        prompt = prompt.replace("[KEYWORDS]", ", ".join(keywords))
    
    if "[DOCUMENTS]" in prompt:
        prompt = replace_documents(prompt, docs)
    
    return prompt

In [20]:
print(create_prompt(docs, topic, topics_ctfidf, prompt))

Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Scienc

In [21]:
prompt_2 = '''Now, look at the Themes and Overarching themes you gave me. Modify/add/remove the themes that are very similar to each other, ones that are too specific and long (e.g. "Serine Theonine Protein Kinase"), be concise in theme generation (e.g. instead of "Income Prediction" say "Income", "Demographic Information"->"Demographics", "Myth-busting"->"Myth Busting", "Travel Industry" -> "Travel" etc.). Also, remove themes that are related to the technical description of a text, e.g. "Regression", "Numerical Features", "Data Analysis", "Datasets", "Data", "Prediction", "Data Science", etc.. Instead, we care about keeping descriptive, interpretable, coherent themes. Same JSON format.'''

In [22]:
top_k = 5
top_k_topics = []
for i in range(len(data)):
    top_k_indices = np.argsort(probs[i])[::-1][:top_k]
    top_k_indices = [index for index in top_k_indices]
    top_k_indices = [index for index in top_k_indices if index != -1]
    top_k_topics.append([str(topic) for topic in top_k_indices])

top_k_topics[0]

['315', '343', '920', '559', '332']

In [23]:
bottom_k = 1
bottom_k_topics = []
for i in range(len(data)):
    bottom_k_indices = np.argsort(probs[i])[:bottom_k]
    bottom_k_indices = [index for index in bottom_k_indices]
    bottom_k_indices = [index for index in bottom_k_indices if index != -1]
    bottom_k_topics.append([str(topic) for topic in bottom_k_indices])

bottom_k_topics[0]

['424']

In [24]:
def split_themes(data):
    themes_dict = {}
    overarching_themes_dict = {}
    
    for key, value in data['topic_labels'].items():
        # Split the value into themes and overarching themes
        # print(key) 
        themes_part, overarching_part = value.split("; Overarching themes: ")
        
        # Further split to remove the leading key part and "Themes: " label
        themes = themes_part.split("_Themes: ")[1]
        
        # Store in dictionaries
        themes_dict[key] = themes.strip()
        overarching_themes_dict[key] = overarching_part.strip()
    
    return themes_dict, overarching_themes_dict

themes_dict, overarching_themes_dict = split_themes(topics_json)

# Print results (for demonstration purposes)
print("Themes Dictionary:")
for k, v in themes_dict.items():
    print(f"{k}: {v}")

print("\nOverarching Themes Dictionary:")
for k, v in overarching_themes_dict.items():
    print(f"{k}: {v}")

Themes Dictionary:
0: Binarized Regression Problem, Chemistry, Life Science, Mythbusting, Study, Target Feature, Nominal Classification
1: Anal Cat Data, Binarization, Chemistry, Life Science, Myth Busting, Study
2: COVID-19, Pandemic, Cases, Deaths, Data Analysis, Visualization, Vaccinations, Demographics, Economics, Machine Learning, Statistics, Mobility, Tests, Healthcare, Length of Stay, Safety Behavior, No-Show Problem, Education Disruption
3: analcatdata, datasets, book, education, research, statistics, manufacturing, automotive accessories, hardware, bolts, counting, electronic eye, sensitivity setting, machine adjustments, time, data formats, comma-delimited, Microsoft Excel, quotation marks, backslashes, blanks, space replacement, acknowledgement, scientific, educational, non-commercial purposes, analytical research, mythbusting, study, hotel, luxury, Europe, customer reviews, scoring, geographical location, sentiment analysis, recommendation engine, clustering, visualization,

# TEMPORARY

In [25]:
# loaded_model.get_topic(6)

[[{'Themes': [['QSAR', 1],
    ['Drug Target', 1],
    ['Serine Threonine Protein Kinase', 1],
    ['Molecular Fingerprints', 1],
    ['MetaQSAR Project', 1],
    ['Protein Phosphorylation', 1],
    ['Autophagy', 1],
    ['Cell Cycle', 1],
    ['Signal Transduction', 1],
    ['Apoptotic Process', 1],
    ['Myosin', 1],
    ['ULK', 1],
    ['Citron', 1],
    ['Nek', 1],
    ['Rho', 1],
    ['Tousled', 1]],
   'Overarching themes': [['Biology', 1],
    ['Chemistry', 1],
    ['Pharmacology', 1],
    ['Health', 1]]},
  1]]

In [ ]:
# loaded_model.get_document_info(data).loc[0]

Document                   Anneal \nTags: Manufacturing, Study One, Study...
Topic                                                                    489
Name                       489_Themes: Annealing, Steel, Material Science...
Representation             [Themes: Annealing, Steel, Material Science, M...
Representative_Docs                                                      NaN
Top_n_words                Themes: Annealing, Steel, Material Science, Me...
Representative_document                                                False
Name: 0, dtype: object

In [ ]:
# with open('results.json') as f:
#     results = json.load(f)

# with open('overarching_results.json') as f:
#     overarching_results = json.load(f)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# # from tqdm import tqdm

# # def filter_similar_labels(labels, scores, threshold = 0.95):
# #     embeddings = embedding_model.encode(labels)
    
# #     cosine_similarities = cosine_similarity(embeddings)
    
# #     similar_labels = set()
# #     for i in range(len(labels)):
# #         for j in range(i + 1, len(labels)):
# #             if cosine_similarities[i, j] > threshold:
# #                 if scores[i] > scores[j]:
# #                     similar_labels.add(labels[j])
# #                 else:
# #                     similar_labels.add(labels[i])
    
# #     return [label for label in labels if label not in similar_labels], [score for label, score in zip(labels, scores) if label not in similar_labels]

# # def filter_results(results, score_threshold=0.5):
# #     filtered_results = []

# #     for result in tqdm(results):
# #         filtered = [(label, score) for label, score in zip(result['labels'], result['scores']) if score > score_threshold]

# #         if filtered:
# #             labels, scores = zip(*filtered)
# #             filtered_labels, filtered_scores = filter_similar_labels(list(labels), list(scores))
# #         else:
# #             filtered_labels, filtered_scores = [], []

# #         filtered_results.append({
# #             'sequence': result['sequence'],
# #             'labels': filtered_labels,
# #             'scores': filtered_scores
# #         })

# #     return filtered_results

In [ ]:
# df_results = pd.DataFrame(filter_results(results))
# df_results

In [ ]:
# df_overarching_results = pd.DataFrame(filter_results(overarching_results))
# df_overarching_results

In [ ]:
# df_results.to_csv('filtered_results.csv', index=False)
# df_overarching_results.to_csv('filtered_overarching_results.csv', index=False)

In [ ]:
# df_results = pd.read_csv('filtered_results.csv')
# df_results

,sequence,labels,scores
0,"Anneal \nTags: Manufacturing, Study One, Study...","['Anneal', 'Steel', 'Data Sets', 'Alloy', 'Met...","[0.9781602621078491, 0.944840133190155, 0.7416..."
1,"Anneal \nTags: Manufacturing, Study One, Study...",['Studies'],[0.6087751984596252]
2,"Anneal \nTags: Manufacturing, Study One, Study...","['Anneal', 'Steel', 'Alloy', 'Metallurgy', 'Ma...","[0.9781602621078491, 0.944840133190155, 0.7214..."
3,**Chess End-Game - King + Rook versus King + P...,"['Chess End-Game', 'Chess', 'King + Rook vs. K...","[0.9902561902999878, 0.954011082649231, 0.9341..."
4,**Labor Negotiations**\n\nThis dataset pertain...,['Canada'],[0.6595309972763062]
...,...,...,...
5280,Credit Score Classification \n\nTags: Person C...,"['Credit', 'Finance', 'Loan']","[0.9906300902366638, 0.664585530757904, 0.5735..."
5281,The One Thirty Two dataset has been tagged wit...,['Ada'],[0.5196735858917236]
5282,Login Data for Cybersecurity Events \n\nTags: ...,"['Login Data for Cybersecurity Event', 'Comput...","[0.9616353511810303, 0.5064505338668823]"
5283,Dataset Name: User Ravi Tags: Dummy Data \n\nF...,[],[]


In [ ]:
# df_overarching_results = pd.read_csv('filtered_overarching_results.csv')
# df_overarching_results

,sequence,labels,scores
0,"Anneal \nTags: Manufacturing, Study One, Study...","['Manufacturing', 'Science', 'Materials', 'Mat...","[0.9330682158470154, 0.7778840661048889, 0.760..."
1,"Anneal \nTags: Manufacturing, Study One, Study...",['Science'],[0.7778840661048889]
2,"Anneal \nTags: Manufacturing, Study One, Study...","['Manufacturing', 'Material', 'Science', 'Mate...","[0.9330682158470154, 0.8098275661468506, 0.777..."
3,**Chess End-Game - King + Rook versus King + P...,"['Chess', 'Research']","[0.954011082649231, 0.6249885559082031]"
4,**Labor Negotiations**\n\nThis dataset pertain...,[],[]
...,...,...,...
5280,Credit Score Classification \n\nTags: Person C...,['Finance'],[0.664585530757904]
5281,The One Thirty Two dataset has been tagged wit...,['Technology'],[0.8180697560310364]
5282,Login Data for Cybersecurity Events \n\nTags: ...,"['Technology', 'Security', 'Computer']","[0.9751905798912048, 0.9700767993927002, 0.659..."
5283,Dataset Name: User Ravi Tags: Dummy Data \n\nF...,"['Technology', 'Computer']","[0.8555715084075928, 0.5300011038780212]"


In [ ]:
# # find embedding similarity between "computer science" and "computational science"
# embedding_model.encode(["computer science", "computational science"])

# # similarity
# cosine_similarity([embedding_model.encode("computer science")], [embedding_model.encode("computational science")])

array([[0.95230937]], dtype=float32)

In [ ]:
# cosine_similarity([embedding_model.encode("Financial market")], [embedding_model.encode("Finance")])

array([[0.89755094]], dtype=float32)

In [ ]:
# cosine_similarity([embedding_model.encode("Used Car")], [embedding_model.encode("Car")])

array([[0.85056174]], dtype=float32)

In [ ]:
# cosine_similarity([embedding_model.encode("Analcatdata")], [embedding_model.encode("Anal Cat Data")])

array([[0.88096756]], dtype=float32)

# Continue here

In [ ]:
# df_datasets = pd.read_pickle("data/datasets.pkl")
# df_datasets

,ignore_attribute,dataset_id,name,version,description,cache_format,format,creator,contributor,collection_date,...,parquet_file,_dataset,_parquet_url,_features,_qualities,_no_qualities_found,data_pickle_file,data_feather_file,feather_attribute_file,openml_url
0,None,2,anneal,1,**Author**: Unknown. Donated by David Sterling...,pickle,ARFF,"[David Sterling, Wray Buntine]",David Sterling and Wray Buntine,1990,...,None,None,http://openml1.win.tue.nl/dataset2/dataset_2.pq,"{0: [0 - family (nominal)], 1: [1 - product-ty...",None,False,None,None,None,https://www.openml.org/d/2
1,None,3,kr-vs-kp,1,Author: Alen Shapiro\nSource: [UCI](https://ar...,pickle,ARFF,Alen Shapiro,Rob Holte,1989-08-01,...,None,None,http://openml1.win.tue.nl/dataset3/dataset_3.pq,"{0: [0 - bkblk (nominal)], 1: [1 - bknwy (nomi...",None,False,None,None,None,https://www.openml.org/d/3
2,None,4,labor,1,**Author**: Unknown\n**Source**: Collective Ba...,pickle,ARFF,Collective Bargaining Review of Labour Canada,Stan Matwin,1988-11-01,...,None,None,http://openml1.win.tue.nl/dataset4/dataset_4.pq,"{0: [0 - duration (numeric)], 1: [1 - wage-inc...",None,False,None,None,None,https://www.openml.org/d/4
3,None,5,arrhythmia,1,"**Author**: H. Altay Guvenir, Burak Acar, Hald...",pickle,ARFF,"[H. Altay Guvenir, Burak Acar, Haldun Muderris...",None,1998-01-01,...,None,None,http://openml1.win.tue.nl/dataset5/dataset_5.pq,"{0: [0 - age (numeric)], 1: [1 - sex (nominal)...",None,False,None,None,None,https://www.openml.org/d/5
4,None,6,letter,1,**Author**: David J. Slate \n**Source**: [UCI...,pickle,ARFF,David J. Slate,None,1991-01-01,...,None,None,http://openml1.win.tue.nl/dataset6/dataset_6.pq,"{0: [0 - x-box (numeric)], 1: [1 - y-box (nume...",None,False,None,None,None,https://www.openml.org/d/6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5674,None,46204,US-Births,1,Number of births in the United States.\n\nFrom...,pickle,arff,"Randall Pruim, Daniel Kaplan, Nicholas Horton",Bruno Belucci,1968 - 1988,...,None,None,https://openml1.win.tue.nl/datasets/0004/46204...,"{0: [0 - id_series (nominal)], 1: [1 - date (s...",None,False,None,None,None,https://www.openml.org/d/46204
5675,None,46205,Solar-Power,1,Solar power production in MW recorded per ever...,pickle,arff,"Godahewa, Rakshitha; Bergmeir, Christoph; Web...",None,2020,...,None,None,https://openml1.win.tue.nl/datasets/0004/46205...,"{0: [0 - id_series (nominal)], 1: [1 - value_0...",None,False,None,None,None,https://www.openml.org/d/46205
5676,None,46206,Wind-Power,1,Wind power production in MW recorded per every...,pickle,arff,"Godahewa, Rakshitha; Bergmeir, Christoph; Web...",None,2020,...,None,None,https://openml1.win.tue.nl/datasets/0004/46206...,"{0: [0 - id_series (nominal)], 1: [1 - value_0...",None,False,None,None,None,https://www.openml.org/d/46206
5677,None,46207,Wind-Power,2,Wind power production in MW recorded per every...,pickle,arff,"Godahewa, Rakshitha; Bergmeir, Christoph; Web...",None,2020,...,None,None,https://openml1.win.tue.nl/datasets/0004/46207...,"{0: [0 - id_series (nominal)], 1: [1 - value_0...",None,False,None,None,None,https://www.openml.org/d/46207


In [73]:
texts = [
    "Labor Dataset",
    "Alcohol Consumption in Russia",
    "GameStop Historical Stock Prices",
    "COVID-19 Rio de Janeiro",
    "FOREX USD/JPY Hourly Close Data",
    "Wine Dataset\n\nTags: Partial Label Ranking",
    "Baseball Hitter",
    "Arrhythmia Dataset",
    "Breast Cancer Dataset\n\nTags: mythbusting",
    "League of Legends Solo Queue",
    "Popular Movies of IMDb",
    "QSAR-TID-12744",
]
overlapping_texts = [
    ("League of Legends Solo Queue", "Esports Matches in 2020"),
    ("Popular Movies of IMDb", "Movies on Netflix, Prime Video, Hulu, and Disney"),
    ("QSAR-TID-12744", "QSAR-TID-10031"),
    ("Labor Dataset", "Name: Strikes\nTags: OpenML"),
    ("COVID-19 Rio de Janeiro", "COVID-19 Biotech Companies on Stock Exchange"),
    ("Breast Cancer Dataset\n\nTags: mythbusting", "Lung Dataset, a microarray"),
]

In [74]:
import json

def extract_labels_and_scores(df, texts, overlapping_texts):
    results = {}

    all_texts = texts + [item for pair in overlapping_texts for item in pair]
    unique_texts = list(set(all_texts))

    for text in unique_texts:
        try:
            # Find the corresponding entry in the DataFrame
            entry = df.loc[df['prompt_description'].str.contains(text, case=False, na=False)].values[0][0]
            
            # Parse the JSON string
            data = json.loads(entry)
            
            # Extract labels and scores
            labels = data[0]['labels']
            scores = data[0]['scores']
            
            results[text] = {'labels': labels, 'scores': scores}
        
        except IndexError:
            print(f"No matching entry found for: {text}")
        except json.JSONDecodeError:
            print(f"Error decoding JSON for entry: {text}")
        except Exception as e:
            print(f"Error processing entry {text}: {str(e)}")

    return results

# Assuming df is your DataFrame
results = extract_labels_and_scores(df, texts, overlapping_texts)

# Print results
for text, data in results.items():
    print(f"\nResults for: {text}")
    print(f"Labels: {data['labels']}")
    print(f"Scores: {data['scores']}")

Error decoding JSON for entry: Name: Strikes
Tags: OpenML
Error decoding JSON for entry: Breast Cancer Dataset

Tags: mythbusting
Error decoding JSON for entry: QSAR-TID-12744
Error decoding JSON for entry: COVID-19 Biotech Companies on Stock Exchange
Error decoding JSON for entry: FOREX USD/JPY Hourly Close Data
Error decoding JSON for entry: Popular Movies of IMDb
Error decoding JSON for entry: Labor Dataset
Error decoding JSON for entry: Arrhythmia Dataset
Error decoding JSON for entry: Lung Dataset, a microarray
Error decoding JSON for entry: Wine Dataset

Tags: Partial Label Ranking
Error decoding JSON for entry: Baseball Hitter
Error decoding JSON for entry: QSAR-TID-10031
Error decoding JSON for entry: Alcohol Consumption in Russia
Error decoding JSON for entry: Esports Matches in 2020
Error decoding JSON for entry: Movies on Netflix, Prime Video, Hulu, and Disney
Error decoding JSON for entry: COVID-19 Rio de Janeiro
Error decoding JSON for entry: League of Legends Solo Queue
E

In [71]:
print(df.loc[df['prompt_description'].str.contains('Labor Dataset')].values[0][0])

Labor Dataset

Tags: mythbusting, study, test, uci

Source: Collective Bargaining Review, Labour Canada

Please cite: https://archive.ics.uci.edu/ml/citation_policy.html

Title: Final settlements in labor negotiations in Canadian industry

The dataset includes all collective agreements reached in the business and personal services sector for locals with at least 500 members (teachers, nurses, university staff, police, etc.) in Canada in 1987 and the first quarter of 1988. The data was sourced from the Collective Bargaining Review, a monthly publication by Labour Canada's Industrial Relations Information Service in Ottawa, Ontario.

This dataset has been used for testing concept learning software, particularly an experimental method to learn two-tiered concept descriptions. The data was utilized to learn the description of acceptable and unacceptable contracts. Unacceptable contracts were either obtained through expert interviews or by inventing near misses. The data was also employed t

## Overlap

# Get keywords for each question

In [46]:
hypothesis_template = "This text is about {}"

In [47]:
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
zeroshot_model = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-base-zeroshot-v1.1-all-33", hypothesis_template=hypothesis_template, multi_label=True)

In [48]:
len(tokenizer("GameStop Historical Stock Prices\n\nTags: GameStop Corporation is an American video game, consumer electronics, and gaming merchandise retailer.\n\nGameStop's stocks have experienced a remarkable surge in recent days, largely attributed to the sudden popularity of a Reddit group, which has blown up the stock price. As a result, GameStop is now up significantly since the start of the year, making it an intriguing case study to predict the stock prices in the next few months and assess the performance of prediction and time series models.\n\nThis dataset comprises daily historical stock prices of GameStop from the year it was listed, February 2002, to January 2021.\n\nThe dataset contains the following information:\n\nDate: The date of trading\nOpen Price: The opening price of the stock\nHigh Price: The highest price of that day\nLow Price: The lowest price of that day\nClose Price: The closed price of that day\nVolume: The amount of stocks traded during that day\nAdjusted Close Price: The stock's closing price has been amended to include any distributions or corporate actions that occur before the next day's open.\n\nAcknowledgements\nThis dataset was collected from the Yahoo Finance website.\n\nInspiration\nHow does a sudden increase in volume affect stock trading? What is the impact of the variance between the adjusted close and the next day's opening price?\n\nKeywords: GameStop, stock prices, Reddit, time series models, prediction, trading volume, adjusted close price.")['input_ids'])

277

In [ ]:
# load json from results_llama_deberta_v2.json
import json

with open('results_llama_deberta_v2.json') as f:
    results = json.load(f)

with open('overarching_results_llama_deberta_v2') as f:
    overarching_results = json.load(f)

with open('results_llama_deberta_keywords_v2.json') as f:
    keywords_results = json.load(f)

In [49]:
text = "GameStop Historical Stock Prices\n\nTags: GameStop Corporation is an American video game, consumer electronics, and gaming merchandise retailer.\n\nGameStop's stocks have experienced a remarkable surge in recent days, largely attributed to the sudden popularity of a Reddit group, which has blown up the stock price. As a result, GameStop is now up significantly since the start of the year, making it an intriguing case study to predict the stock prices in the next few months and assess the performance of prediction and time series models.\n\nThis dataset comprises daily historical stock prices of GameStop from the year it was listed, February 2002, to January 2021.\n\nThe dataset contains the following information:\n\nDate: The date of trading\nOpen Price: The opening price of the stock\nHigh Price: The highest price of that day\nLow Price: The lowest price of that day\nClose Price: The closed price of that day\nVolume: The amount of stocks traded during that day\nAdjusted Close Price: The stock's closing price has been amended to include any distributions or corporate actions that occur before the next day's open.\n\nAcknowledgements\nThis dataset was collected from the Yahoo Finance website.\n\nInspiration\nHow does a sudden increase in volume affect stock trading? What is the impact of the variance between the adjusted close and the next day's opening price?\n\nKeywords: GameStop, stock prices, Reddit, time series models, prediction, trading volume, adjusted close price."

In [50]:
labels = [
    "Stock",
    "Stock Market",
    "Financial Metrics",
    "Stock Prices",
    "Financial Data",
    "Price",
    "Finance",
    "Stock Price",
    "Financial Analysis",
    "Price Data",
    "Publicly Traded Companies",
    "Stock Data",
    "Investments",
    "Historical Price Data",
    "Stock Trading",
    "Companies",
    "Investment",
    "Trading",
    "Market Analysis",
    "Daily Prices",
    "Historical Stock Prices",
    "GameStop",
    "Historical Data",
    "Fundamental Analysis",
    "Daily Trading Data",
    "Equity Shares",
    "Numerai",
    "Market Capitalization",
    "Volatility",
    "Company Performance",
    "Market Indicators",
    "Technical Analysis",
    "Company Information",
    "American Companies",
    "Wall Street",
    "Information",
    "Data Analytics",
    "Market Conditions",
    "Earnings",
    "Financial Modeling",
    "Dataset Analysis",
    "Fundamental Statistics",
    "High",
    "Stock Market Index",
    "Global Equity",
    "Time-Series Analysis",
    "Indicators",
    "Time Series Analysis",
    "Close Price",
    "Ask",
    "Forecasting",
    "Predictive Modeling",
    "Trading Volume",
    "Economic Metrics",
    "Price Prediction",
    "Open Price",
    "Volume Traded",
    "Change",
    "Modeling",
    "Investment Option",
    "Sentiment Analysis",
    "Predictions",
    "Close",
    "Volume",
    "Time Series Models",
    "Trade Strategies",
    "Investment Opportunities",
    "Company Portfolio",
    "Stock Exchange",
    "Data Enrichment",
    "Equity Symbols",
    "Momentum Strategy",
    "Stock Listings",
    "Time Series",
    "Categorical Features",
    "Macd",
    "Financial Ratios",
    "Economic Indicator",
    "Benchmarking",
    "Trading Name",
    "Options Trading",
    "Numerical Features",
    "Wingstop Stock Prices",
    "Binary Classification",
    "Industries",
    "Interval",
    "Algorithm Training",
    "Diversified Investments",
    "Return on Equity",
    "Day",
    "Class Attribute",
    "Yahoo Finance",
    "Reddit",
    "Aud/SGD",
    "Nse",
    "Date Range",
    "Return on Assets",
    "Currency",
    "New York Stock Exchange",
    "Sepal Length",
    "Backtest",
    "Algorithm Selection",
    "Mean Close Price",
    "Compas",
    "Candlestick",
    "Internet",
    "Hashtags",
    "Banking Data",
    "Low",
    "Price-to-Earnings Ratio",
    "Futures",
    "Social Media",
    "Regression Dataset",
    "Aud/Usd",
    "Conflict-Driven Learning",
    "E-Commerce",
    "Open Interest",
    "Nasdaq 100",
    "CSV File",
    "Aud",
    "Support and Resistance",
    "Dukascopy",
    "Technology Corporation",
    "Bid and Ask Prices",
    "Banking",
    "National Stock Exchange",
    "Feature Engineering",
    "Nasdaq",
    "Google Stock Data",
    "Economic Turbulence",
    "Foreign Exchange",
    "Hang Seng",
    "Bid",
    "Minute Close",
    "December",
    "Forex",
    "Yahoo Finance API",
    "Faang",
    "Cad/CHF",
    "Answer Set Programming",
    "Natural Language Processing",
    "Profit to Equity Ratio",
    "Long Position",
    "Usd",
    "Month Information",
    "Public Sentiment",
    "Corporate Actions",
    "Moving Median",
    "Ibov",
    "Competition",
    "Aud/Jpy",
    "Iris",
    "Antm Jk",
    "Nifty Fifty Index",
    "Credit Analysis",
    "Personal Finance",
    "Rumors",
    "Minute",
    "Mathematica",
    "Profit to Book Value Ratio",
    "Python",
    "Encrypted Data",
    "Image Analysis",
    "Cad",
    "Eur/Aud",
    "ASP Solver",
    "Petal Width",
    "Credit",
    "Minute Interval",
    "Cad/Jpy",
    "Charge Degree",
    "AAPL Stock Price",
    "Dow Jones",
    "SPY Index",
    "Influencers",
    "United States Dollar",
    "Tip Amount",
    "Initial Public Offering",
    "Amazon Stock Price",
    "Geographic Analysis",
    "Subscription",
    "Green Line",
    "Dow Jones Index",
    "Telecom Company",
    "Ionosphere",
    "NYC Taxi",
    "SGD",
    "Dividend Yield",
    "Apple Stock",
    "Aud/Cad",
    "Eur/Usd",
    "HDFC Bank",
    "Jpy",
    "Payment Type",
    "Google",
    "Eur/Huf",
    "Apple Watch",
    "New York Mercantile Exchange",
    "GBP/Usd",
    "Reliance Industries",
    "GitHub Repository",
    "Electric Vehicle",
    "Hour",
    "Eur/Sek",
    "Dow Jones Industrial Average",
    "Battery Factory",
    "Tamil Nadu",
    "Eur Jpy",
    "Alphabet",
    "CHF/SGD",
    "Eur/HKD",
    "Demographic Data",
    "NZD/Usd",
    "Eur/Cad",
    "NZD",
    "Canadian Dollar",
    "Tweets",
    "Location ID",
    "Eur/NZD",
    "Ram",
    "Eur/GBP",
    "Criminal Justice",
    "Sensex",
    "Amazon",
    "Eur/DKK",
    "CHF",
    "Bank",
    "Satellite Imagery",
    "Twitter",
    "Crude Oil",
    "Indian Stock Exchange",
    "Tesla",
    "Dutch",
    "New York City",
    "Bombay Stock Exchange",
    "Netflix",
    "Aerospace Companies",
    "GBP",
    "Singapore Dollar",
    "Jio Platform",
    "Day of the Week",
    "Capitol Riot",
    "Indonesia",
    "Microsoft",
    "Taxi",
    "Recession",
    "Currency Pair",
    "Demographics",
    "Apple",
    "Trump",
    "Timezone",
    "iPad",
    "Europe/Amsterdam",
    "Gold",
    "Facebook",
    "Hong Kong",
    "iPhone",
    "Tweepy",
    "Amsterdam",
    "Presidential Election",
    "Indian Economy",
    "Euro",
    "India",
    "Nickel",
    "Recidivism",
    "Environmental Social and Governance",
    "Europe",
    "Computer Software",
    "Species",
    "Trip",
    "Maximum Pain",
    "Remote Sensing",
    "Education",
    "Speeches",
]

In [51]:
labels_overarching = [
    "Stock Market",
    "Finance",
    "Investment",
    "Business",
    "Economics",
    "Computing",
    "Economy",
    "Social Media",
    "Technology",
    "Transportation",
    "Geography",
    "Language",
    "Government",
    "Justice",
    "Elections",
    "Politics",
    "Biology",
]

In [52]:
labels_keywords = [
    "Stock Prices",
    "Gamestop",
    "Prediction",
    "Adjusted Close Price",
    "Reddit",
    "Time Series Models",
    "Trading Volume",
]

In [53]:
result = zeroshot_model(text, labels); result

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': "GameStop Historical Stock Prices\n\nTags: GameStop Corporation is an American video game, consumer electronics, and gaming merchandise retailer.\n\nGameStop's stocks have experienced a remarkable surge in recent days, largely attributed to the sudden popularity of a Reddit group, which has blown up the stock price. As a result, GameStop is now up significantly since the start of the year, making it an intriguing case study to predict the stock prices in the next few months and assess the performance of prediction and time series models.\n\nThis dataset comprises daily historical stock prices of GameStop from the year it was listed, February 2002, to January 2021.\n\nThe dataset contains the following information:\n\nDate: The date of trading\nOpen Price: The opening price of the stock\nHigh Price: The highest price of that day\nLow Price: The lowest price of that day\nClose Price: The closed price of that day\nVolume: The amount of stocks traded during that day\nAdjusted 

In [54]:
result_overarching = zeroshot_model(text, labels_overarching); result_overarching

{'sequence': "GameStop Historical Stock Prices\n\nTags: GameStop Corporation is an American video game, consumer electronics, and gaming merchandise retailer.\n\nGameStop's stocks have experienced a remarkable surge in recent days, largely attributed to the sudden popularity of a Reddit group, which has blown up the stock price. As a result, GameStop is now up significantly since the start of the year, making it an intriguing case study to predict the stock prices in the next few months and assess the performance of prediction and time series models.\n\nThis dataset comprises daily historical stock prices of GameStop from the year it was listed, February 2002, to January 2021.\n\nThe dataset contains the following information:\n\nDate: The date of trading\nOpen Price: The opening price of the stock\nHigh Price: The highest price of that day\nLow Price: The lowest price of that day\nClose Price: The closed price of that day\nVolume: The amount of stocks traded during that day\nAdjusted 

In [55]:
result_keywords = zeroshot_model(text, labels_keywords); result_keywords

{'sequence': "GameStop Historical Stock Prices\n\nTags: GameStop Corporation is an American video game, consumer electronics, and gaming merchandise retailer.\n\nGameStop's stocks have experienced a remarkable surge in recent days, largely attributed to the sudden popularity of a Reddit group, which has blown up the stock price. As a result, GameStop is now up significantly since the start of the year, making it an intriguing case study to predict the stock prices in the next few months and assess the performance of prediction and time series models.\n\nThis dataset comprises daily historical stock prices of GameStop from the year it was listed, February 2002, to January 2021.\n\nThe dataset contains the following information:\n\nDate: The date of trading\nOpen Price: The opening price of the stock\nHigh Price: The highest price of that day\nLow Price: The lowest price of that day\nClose Price: The closed price of that day\nVolume: The amount of stocks traded during that day\nAdjusted 

In [56]:
result_merged = {
    "labels": result['labels'] + result_overarching['labels'] + result_keywords['labels'],
    "scores": result['scores'] + result_overarching['scores'] + result_keywords['scores']
}

In [57]:
# print lowest score + its label
min_score = min(result_merged['scores'])
min_score_index = result_merged['scores'].index(min_score)
min_score_label = result_merged['labels'][min_score_index]
min_score, min_score_label

(3.477484278846532e-05, 'Politics')

In [58]:
result_merged_filtered = {'labels': [], 'scores': []}

for label, score in zip(result_merged['labels'], result_merged['scores']):
    if score >= 0.9:
        print(f"{label}: {score}")
        result_merged_filtered['labels'].append(label)
        result_merged_filtered['scores'].append(score)

Stock: 0.9997363090515137
Stock Market: 0.9996795654296875
Publicly Traded Companies: 0.999662458896637
Stock Prices: 0.9996222853660583
Financial Data: 0.9995702505111694
Financial Metrics: 0.9995583295822144
Stock Price: 0.9995200634002686
Price Data: 0.9995040893554688
Price: 0.9994812607765198
Finance: 0.9994399547576904
Stock Data: 0.9993910789489746
Companies: 0.9993446469306946
Financial Analysis: 0.9993292689323425
Historical Price Data: 0.9990816116333008
Stock Trading: 0.9989397525787354
GameStop: 0.9989174604415894
Historical Stock Prices: 0.998801589012146
Investments: 0.9986225366592407
Investment: 0.9986112117767334
Market Analysis: 0.9985486268997192
Daily Prices: 0.9984124302864075
Historical Data: 0.998389482498169
Daily Trading Data: 0.998348593711853
Market Capitalization: 0.9983121156692505
Trading: 0.9982667565345764
American Companies: 0.99813312292099
Numerai: 0.9980804324150085
Equity Shares: 0.9978904128074646
Fundamental Analysis: 0.9978018403053284
Market Ind

In [59]:
result_merged_filtered

{'labels': ['Stock',
  'Stock Market',
  'Publicly Traded Companies',
  'Stock Prices',
  'Financial Data',
  'Financial Metrics',
  'Stock Price',
  'Price Data',
  'Price',
  'Finance',
  'Stock Data',
  'Companies',
  'Financial Analysis',
  'Historical Price Data',
  'Stock Trading',
  'GameStop',
  'Historical Stock Prices',
  'Investments',
  'Investment',
  'Market Analysis',
  'Daily Prices',
  'Historical Data',
  'Daily Trading Data',
  'Market Capitalization',
  'Trading',
  'American Companies',
  'Numerai',
  'Equity Shares',
  'Fundamental Analysis',
  'Market Indicators',
  'Wall Street',
  'Market Conditions',
  'Company Performance',
  'Data Analytics',
  'Company Information',
  'Technical Analysis',
  'Volatility',
  'Information',
  'Earnings',
  'Fundamental Statistics',
  'Stock Market Index',
  'Dataset Analysis',
  'Time-Series Analysis',
  'Financial Modeling',
  'Time Series Analysis',
  'Global Equity',
  'Trading Volume',
  'Predictive Modeling',
  'Indicato

In [67]:
def analyze_text_with_labels(text, labels, labels_overarching, labels_keywords):
    # Combine all labels into one list
    all_labels = labels + labels_overarching + labels_keywords
    
    # Single call to zeroshot_model
    result = zeroshot_model(text, all_labels)
    
    # Find lowest score and its label
    min_score = min(result['scores'])
    min_score_index = result['scores'].index(min_score)
    min_score_label = result['labels'][min_score_index]
    
    # Filter results with score >= 0.9
    result_filtered = {'labels': [], 'scores': []}
    for label, score in zip(result['labels'], result['scores']):
        if score >= 0.9:
            print(f"{label}: {score}")
            result_filtered['labels'].append(label)
            result_filtered['scores'].append(score)
    
    return result_filtered, (min_score, min_score_label)

# Example usage:
filtered_results, (lowest_score, lowest_score_label) = analyze_text_with_labels(text, labels, labels_overarching, labels_keywords)

Stock: 0.9997363090515137
Stock Market: 0.9996795654296875
Stock Market: 0.9996795654296875
Publicly Traded Companies: 0.999662458896637
Stock Prices: 0.9996222853660583
Stock Prices: 0.9996222853660583
Financial Data: 0.9995702505111694
Financial Metrics: 0.9995583295822144
Stock Price: 0.9995200634002686
Price Data: 0.9995040893554688
Price: 0.9994812607765198
Finance: 0.9994399547576904
Finance: 0.9994399547576904
Stock Data: 0.9993910789489746
Companies: 0.9993446469306946
Financial Analysis: 0.9993292689323425
Historical Price Data: 0.9990816116333008
Gamestop: 0.9989816546440125
Stock Trading: 0.9989397525787354
GameStop: 0.9989174604415894
Historical Stock Prices: 0.998801589012146
Investments: 0.9986225366592407
Investment: 0.9986112117767334
Investment: 0.9986112117767334
Market Analysis: 0.9985486268997192
Daily Prices: 0.9984124302864075
Historical Data: 0.998389482498169
Daily Trading Data: 0.998348593711853
Market Capitalization: 0.9983121156692505
Trading: 0.9982667565345

In [60]:
small_embedding_model = SentenceTransformer('all-mpnet-base-v2')

In [61]:
merged_labels = list(set(result_merged_filtered))
embeddings = small_embedding_model.encode(merged_labels, show_progress_bar=True)
cosine_sim_matrix = cosine_similarity(embeddings)
df = pd.DataFrame(cosine_sim_matrix, index=merged_labels, columns=merged_labels); df

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,scores,labels
scores,1.000000,0.305464
labels,0.305464,1.000000


In [62]:
# for each label, sum its cosine similarity with all other labels and sort them in separate lists
sums = df.sum(axis=1)
sums = sums.sort_values(ascending=False)
sums

scores    1.305465
labels    1.305465
dtype: float32

In [63]:
example = small_embedding_model.encode(["Price", "Price Data"])
cosine_similarity(example)

array([[1.0000001 , 0.48362225],
       [0.48362225, 1.0000001 ]], dtype=float32)

In [64]:
from collections import Counter

def process_and_merge_labels(data, similarity_threshold=0.48, score_threshold=0.1, verbose=False):
    all_labels = [label for item in data for label in item['labels']]
    unique_labels = list(set(all_labels))
    label_counts = Counter(all_labels)
    embeddings = small_embedding_model.encode(unique_labels, show_progress_bar=True)
    cosine_sim_matrix = cosine_similarity(embeddings)
    cosine_sim_matrix_df = pd.DataFrame(cosine_sim_matrix, index=unique_labels, columns=unique_labels)
    
    for result in data:
        labels = result['labels']
        scores = result['scores']
        current_cosine_sim_matrix = cosine_sim_matrix_df.loc[labels, labels].values
        
        similar_labels = []
        for i, row in enumerate(current_cosine_sim_matrix):
            for j, sim in enumerate(row):
                if i < j and sim > similarity_threshold:
                    similar_labels.append((labels[i], labels[j]))
       
        for label1, label2 in similar_labels:
            if label1 in labels and label2 in labels:
                idx1, idx2 = labels.index(label1), labels.index(label2)
                score1, score2 = scores[idx1], scores[idx2]
                count1, count2 = label_counts[label1], label_counts[label2]
                
                if abs(score1 - score2) <= score_threshold:
                    counts_close = abs(count1 - count2) / max(count1, count2) < 0.6
                    
                    if counts_close:
                        if len(label1) <= len(label2):
                            removed_label, kept_label = label2, label1
                            labels.remove(label2)
                            scores.pop(idx2)
                        else:
                            removed_label, kept_label = label1, label2
                            labels.remove(label1)
                            scores.pop(idx1)
                        if verbose:
                            print(f"Merged '{removed_label}' into '{kept_label}' (close counts, shorter label preferred)")
                    elif count1 > count2:
                        labels.remove(label2)
                        scores.pop(idx2)
                        if verbose:
                            print(f"Merged '{label2}' into '{label1}' (higher count preferred)")
                    else:
                        labels.remove(label1)
                        scores.pop(idx1)
                        if verbose:
                            print(f"Merged '{label1}' into '{label2}' (higher count preferred)")
        
        result['labels'] = labels
        result['scores'] = scores
    for result in data:
        result['labels'], result['scores'] = zip(*sorted(zip(result['labels'], result['scores']), key=lambda x: x[1], reverse=True))
    return result

# Call the function
result_final = process_and_merge_labels([result_merged], verbose=True); result_final

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Merged 'Stock Market' into 'Stock' (close counts, shorter label preferred)
Merged 'Stock Prices' into 'Stock' (close counts, shorter label preferred)
Merged 'Stock Price' into 'Stock' (close counts, shorter label preferred)
Merged 'Stock Data' into 'Stock' (close counts, shorter label preferred)
Merged 'Stock Trading' into 'Stock' (close counts, shorter label preferred)
Merged 'Investments' into 'Stock' (close counts, shorter label preferred)
Merged 'Market Analysis' into 'Stock' (close counts, shorter label preferred)
Merged 'Market Capitalization' into 'Stock' (close counts, shorter label preferred)
Merged 'Equity Shares' into 'Stock' (close counts, shorter label preferred)
Merged 'Stock Market' into 'Stock' (close counts, shorter label preferred)
Merged 'Stock Prices' into 'Stock' (close counts, shorter label preferred)
Merged 'Publicly Traded Companies' into 'Companies' (close counts, shorter label preferred)
Merged 'Financial Metrics' into 'Financial Data' (close counts, shorter l

{'labels': ('Stock',
  'Price',
  'Finance',
  'GameStop',
  'Historical Data',
  'Trading',
  'American Companies',
  'Numerai',
  'Volatility',
  'Information',
  'Fundamental Statistics',
  'Adjusted Close Price',
  'Indicators',
  'Prediction',
  'Economic Metrics',
  'Modeling',
  'Investment Option',
  'Time Series Models',
  'Sentiment Analysis',
  'Company Portfolio',
  'High',
  'Close',
  'Data Enrichment',
  'Ask',
  'Trading Name',
  'Open Price',
  'Macd',
  'Volume Traded',
  'Trade Strategies',
  'Equity Symbols',
  'Categorical Features',
  'Economics',
  'Time Series',
  'Stock Listings',
  'Wingstop Stock Prices',
  'Stock Exchange',
  'Volume',
  'Momentum Strategy',
  'Economic Indicator',
  'Financial Ratios',
  'Industries',
  'Binary Classification',
  'Benchmarking',
  'Change',
  'Class Attribute',
  'Numerical Features',
  'Options Trading',
  'Nse',
  'Diversified Investments',
  'Sepal Length',
  'Algorithm Training',
  'Return on Equity',
  'Futures',
  'In

In [65]:
# get top 7 labels
result_final['labels'][:7]

('Stock',
 'Price',
 'Finance',
 'GameStop',
 'Historical Data',
 'Trading',
 'American Companies')